In [8]:
import mlflow
from mlflow import MlflowClient
import mlflow.sklearn
import pandas as pd


mlflow.set_tracking_uri("file:../mlruns")
EXPERIMENT_NAME = "fraud_detection"
mlflow.set_experiment(EXPERIMENT_NAME)


# --- 2. Récupération de l'expérience et de tous les runs
client = MlflowClient()
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.f1_score DESC"],
    max_results=1
)


# --- 3. Sélection du meilleur run
best_run = runs[0]
best_run_id = best_run.info.run_id
best_f1 = best_run.data.metrics.get("f1_score", None)

print(f"✅ Meilleur run trouvé : {best_run_id}")
print(f"   F1-score : {best_f1:.4f}")


# --- 4. Chargement du modèle complet (pipeline)
model_uri = f"runs:/{best_run_id}/fraud_pipeline"
model = mlflow.sklearn.load_model(model_uri)
print("✅ Modèle chargé avec succès depuis MLflow.")

df_raw = pd.read_csv("/Users/martinper/Downloads/fraudTest_head.csv")

# --- 5. Inférence sur une nouvelle donnée
new_data = df_raw.sample(1)

print(new_data)
prediction = model.predict(new_data)
print("🧾 Prédiction :", prediction)

✅ Meilleur run trouvé : b47a90e6f4634e4ab1ec98017d6776b1
   F1-score : 0.5714
✅ Modèle chargé avec succès depuis MLflow.
   Unnamed: 0.1  Unnamed: 0 trans_date_trans_time            cc_num  \
2             2           2   2020-06-21 12:14:53  3598215285024754   

                               merchant        category    amt   first   last  \
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28  Ashley  Lopez   

  gender  ...      lat     long city_pop                job         dob  \
2      F  ...  40.6729 -73.5365    34496  Librarian, public  1970-10-21   

                          trans_num   unix_time merch_lat merch_long is_fraud  
2  c81755dbbbea9d5c77f094348a7579be  1371816893  40.49581 -74.196111        0  

[1 rows x 24 columns]
🧾 Prédiction : [0]
